<p style="font-family: Arial; font-size:3.0em;color:yellow; font-style:bold"><br>
Linearisation Techinques
</p><br>

<p style="text-align:center; font-family: Arial; font-size:2.00em;color:cyan; font-style:bold"><br>
Dr. Hoa Bui
</p><br>

<p style="text-align:center; font-family: Arial; font-size:1.00em;color:white; font-style:bold">Originally created for: </p>

<p style="text-align:center; font-family: Arial; font-size:1.00em;color:green; font-style:bold">
Mathematical Optimisation: Theory and Application, AMSI Summer School 2026</p>
    

<br>
<br>

**Objective**: We solve the following quadratic problem   
$$
\begin{aligned}
\max \quad & \sum_{i=1}^n\sum_{j=i}^n q_{ij} x_ix_j\\
\text{s.t.} \quad & \sum_{i=1}^n x_i = k\\
& x_i\in \{0,1\}
\end{aligned}
$$

In [1]:
import math
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB
from typing import Tuple
import pandas as pd
import plotly.express as px
from termcolor import colored

# Generate some random data to test

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of nodes
n = 25

# Number of nodes to be selected
k = math.floor(n / 2)

# Generate symmetric matrix Q
matrix = np.random.randint(10, size=(n, n))
matrix = matrix + matrix.T

### We write in a function to generate symmetric matrix for future use

In [ ]:
def create_matrix(n: int) -> np.ndarray:
    """Creates a symmetric matrix of size n x n with random integers.

    Args:
        n: The dimension of the square matrix.

    Returns:
        A symmetric numpy array of shape (n, n) with random integer values.
    """
    matrix = np.random.randint(1, 10, size=(n, n))
    # Make it symmetric
    matrix = (matrix + matrix.T) / 2
    return matrix

# We write different solvers for the quadratic problems
## (QP) Solution with <span style="color:white; background-color:green;">Gurobi quadratic solver</span>.

In [4]:
qp = None
try:
    # Define model object
    qp = Model("quadratic")

    # Don't display Gurobi output
    qp.setParam("OutputFlag", 0)

    # Set variables
    x_qp = qp.addVars(n, vtype=GRB.BINARY, name="xqp")

    # Select exactly k nodes
    qp.addConstr(gp.quicksum(x_qp[i] for i in range(n)) == k)

    # Add objective function (quadratic)
    qp.setObjective(
        gp.quicksum(
            matrix[i, j] * x_qp[i] * x_qp[j] for i in range(n) for j in range(i, n)
        ),
        GRB.MAXIMIZE,
    )
    # Solve
    qp.optimize()

    # print solution
    print(colored("Objective value:", "green", attrs=["bold"]), qp.ObjVal)
    print(colored("Runtime:", "green", attrs=["bold"]), qp.Runtime)
    print(colored("Solution:", "green", attrs=["bold"]), [x_qp[i].X for i in range(n)])

finally:
    if qp:
        # Dispose of the model to free resources
        qp.dispose()

Set parameter Username
Set parameter LicenseID to value 2630841
Academic license - for non-commercial use only - expires 2026-03-04
Set parameter LicenseID to value 2630841
Academic license - for non-commercial use only - expires 2026-03-04
Objective value: 840.0
Runtime: 0.1655139923095703
Solution: [1.0, 0.0, -0.0, 0.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]
Objective value: 840.0
Runtime: 0.1655139923095703
Solution: [1.0, 0.0, -0.0, 0.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]


### We get quadratic solver into a function

In [ ]:
def qp_model(
    n: int,
    matrix: np.ndarray,
    k: int,
    timeout: int = 3600,
) -> Tuple[float, float, np.ndarray, float]:
    """Solves the quadratic problem using Gurobi's native QP solver.

    Args:
        n: The number of variables.
        matrix: The symmetric matrix Q of quadratic coefficients.
        k: The number of nodes to select.
        timeout: The maximum time in seconds for the solver.

    Returns:
        A tuple containing:
            - The objective value of the solution.
            - The solver runtime.
            - A numpy array representing the solution vector.
            - The final MIP gap.
    """

    qp = None
    try:
        # Define model object
        qp = Model("QP")
        qp.setParam("TimeLimit", timeout)
        qp.setParam("OutputFlag", 0)  # suppress output

        # Set variables
        x_qp = qp.addVars(n, vtype=GRB.BINARY, name="xqp")

        # Select exactly k nodes
        qp.addConstr(gp.quicksum(x_qp[i] for i in range(n)) == k)

        # Add objective function
        qp.setObjective(
            gp.quicksum(
                matrix[i, j] * x_qp[i] * x_qp[j] for i in range(n) for j in range(i, n)
            ),
            GRB.MAXIMIZE,
        )

        # Solve the quadratic program
        qp.optimize()

        # get solution
        return qp.ObjVal, qp.Runtime, np.array([v.X for v in x_qp.values()]), qp.MIPGap
    except gp.GurobiError as e:
        print("Error:", e)
        return None, None, None, None
    finally:
        if qp:
            # Dispose of the model to free resources
            qp.dispose()

## <span style="color:white; background-color:green;">First Linear Formulation</span> and its solution.
$$
\begin{aligned}
\max \quad & \sum_{i=1}^n\sum_{j=i}^n q_{ij} y_{ij}\\
\text{s.t.} \quad & \sum_{i=1}^n x_i = k\\
& x_i+x_j-1 \le y_{ij}\\
& y_{i,j} \le x_i\\
& y_{i,j} \le x_j\\
& x_i\in \{0,1\}\\
& y_{i,j} \ge 0.
\end{aligned}
$$

In [ ]:
m = None
try:
    # Setup model
    m = Model("Linear1")

    # Don't display output
    m.setParam("OutputFlag", 0)

    # Set variables
    x = m.addVars(n, vtype=GRB.BINARY, name="x")

    # Set auxiliary variables
    y = m.addVars(
        n,
        n,
        vtype=GRB.CONTINUOUS,
        name="y",
        ub=1,
        lb=0,
    )
    # Add constraints to models m
    # Select exactly k nodes
    m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

    # Add auxiliary constraints (1) where x_i + y_j <= y_ij
    m.addConstrs(x[i] + x[j] - 1 <= y[i, j] for i in range(n) for j in range(i, n))

    # Add auxiliary constraints (2) where y_ij <= x_i for j <= i
    m.addConstrs(y[i, j] <= x[i] for i in range(n) for j in range(i, n))

    # Add auxiliary constraints (3) where y_ij <= x_j for j >= i
    m.addConstrs(y[i, j] <= x[j] for i in range(n) for j in range(i, n))

    # Add objective function
    m.setObjective(
        gp.quicksum(matrix[i, j] * y[i, j] for i in range(n) for j in range(i, n)),
        GRB.MAXIMIZE,
    )

    # Solve
    m.optimize()

    # print solution
    print(colored("Objective value:", "green", attrs=["bold"]), m.ObjVal)
    print(colored("Runtime:", "green", attrs=["bold"]), m.Runtime)
    print(colored("Solution:", "green", attrs=["bold"]), [x[i].X for i in range(n)])
finally:
    if m:
        m.dispose()

Objective value: 840.0
Runtime: 0.07960009574890137
Solution: [1.0, 0.0, 0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 1.0, -0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, -0.0, 1.0]


In [ ]:
def first_linear(
    n: int,
    matrix: np.ndarray,
    k: int,
    timeout: int = 3600,
) -> Tuple[float, float, np.ndarray, float]:
    """Solves the problem using the first standard linearization technique.

    Args:
        n: The number of variables.
        matrix: The symmetric matrix Q of quadratic coefficients.
        k: The number of nodes to select.
        timeout: The maximum time in seconds for the solver.

    Returns:
        A tuple containing:
            - The objective value of the solution.
            - The solver runtime.
            - A numpy array representing the solution vector.
            - The final MIP gap.
    """
    m = None
    try:
        # Setup model
        m = Model("Linear1")
        m.setParam("TimeLimit", timeout)  # time limits
        m.setParam("OutputFlag", 0)  # suppress output

        # Set variables
        x = m.addVars(n, vtype=GRB.BINARY, name="x")

        # Set auxiliary variables
        y = m.addVars(
            [(i, j) for i in range(n) for j in range(i, n)],
            vtype=GRB.CONTINUOUS,
            name="y",
        )

        # Add constraints to models m
        # Select exactly k nodes
        m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

        # Add auxiliary constraints (1) where x_i + y_j <= y_ij
        m.addConstrs(x[i] + x[j] - 1 <= y[i, j] for i in range(n) for j in range(i, n))

        # Add auxiliary constraints (2) where y_ij <= x_i for j <= i
        m.addConstrs(y[i, j] <= x[i] for i in range(n) for j in range(i, n))

        # Add auxiliary constraints (3) where y_ij <= x_j for j >= i
        m.addConstrs(y[i, j] <= x[j] for i in range(n) for j in range(i, n))

        # Add objective function
        m.setObjective(
            gp.quicksum(matrix[i, j] * y[i, j] for i in range(n) for j in range(i, n)),
            GRB.MAXIMIZE,
        )

        # Solve
        m.optimize()

        return m.ObjVal, m.Runtime, np.array([v.X for v in x.values()]), m.MIPGap
    except gp.GurobiError as e:
        print("Error:", e)
        return None, None, None, None
    finally:
        if m:
            # Dispose of the model to free resources
            m.dispose()

## <span style="color:white; background-color:green;">Second Linear Formulation</span> and its solution.
$$
\begin{aligned}
\max \quad \frac{1}{2}&\sum_{i=1}^{n} w_{i}\\
\text{s.t.} \quad & \sum_{i=1}^n x_i = k\\
& w_i \le x_i \cdot \text{ub}_i\\
& w_i \ge x_i \cdot \text{lb}_i\\
& w_i \le \sum_{j=1}^n x_j \cdot q_{ij} - \text{lb}_i(1-x_i)\\
& w_i \ge \sum_{j=1}^n x_j \cdot q_{ij} - \text{ub}_i(1-x_i)\\
& x\in \{0,1\}^n, w_i \in [\text{lb}_i,\text{ub}_i].
\end{aligned}
$$

### Upperbound (ub) and Lowerbound (lb) conditions
1. $\text{ub}_i = k \cdot \max_{j=i,\ldots,n}q_{ij}$;
2. $\text{lb}_i = k \cdot \min_{j=i,\ldots,n}q_{ij}$.

In [8]:
# Get upperbound and lowerbound
ubi = np.array([k * np.max(matrix[i]) for i in range(n)])

lbi = np.array([k * np.min(matrix[i]) for i in range(n)])

# Lowerbound for all variables
lowerbound = min(0, min(lbi))

In [ ]:
g = None
try:
    # Setup model
    g = Model("Linear2")

    # Dont display output
    g.setParam("OutputFlag", 0)

    # Get variables
    x_g = g.addVars(n, vtype=GRB.BINARY, name="x_g")

    # Get auxiliary variables
    w = g.addVars(n, vtype=GRB.CONTINUOUS, name="w")

    # Select exactly k nodes
    g.addConstr(gp.quicksum(x_g[i] for i in range(n)) == k)

    # Constraint for w, x, ub where w_i <= x_i * ubi[i]
    g.addConstrs(w[i] <= x_g[i] * ubi[i] for i in range(n))

    # Constraint for w, x, lb where w_i >= x_i * lbi[i]
    g.addConstrs(w[i] >= x_g[i] * lbi[i] for i in range(n))

    # Constraint for w, x, q, lb where w_i <= sum(x_j * q_ij) - lbi[i] * (1 - x_i)
    g.addConstrs(
        w[i]
        <= gp.quicksum(x_g[j] * matrix[i, j] for j in range(n)) - lbi[i] * (1 - x_g[i])
        for i in range(n)
    )

    # Constraint for w, x, q, ub where w_i >= sum(x_j * q_ij) - ubi[i] * (1 - x_i)
    g.addConstrs(
        w[i]
        >= gp.quicksum(x_g[j] * matrix[i, j] for j in range(n)) - ubi[i] * (1 - x_g[i])
        for i in range(n)
    )

    # Add objective function
    g.setObjective(
        0.5
        * (
            gp.quicksum(w[i] for i in range(n))
            + gp.quicksum(matrix[i, i] * x_g[i] for i in range(n))
        ),
        GRB.MAXIMIZE,
    )

    # Solve
    g.optimize()

    # print solution
    print(colored("Objective value:", "green", attrs=["bold"]), g.ObjVal)
    print(colored("Runtime:", "green", attrs=["bold"]), g.Runtime)
    print(colored("Solution:", "green", attrs=["bold"]), [x_g[i].X for i in range(n)])
finally:
    if g:
        g.dispose()

Objective value: 840.0
Runtime: 0.1769270896911621
Solution: [1.0, -0.0, -0.0, 0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, -0.0, 1.0, 1.0, -0.0, 1.0]


In [ ]:
def second_linear(
    n: int,
    matrix: np.ndarray,
    k: int,
    timeout: int = 3600,
) -> Tuple[float, float, np.ndarray, float]:
    """Solves the problem using the second linearization technique.

    Args:
        n: The number of variables.
        matrix: The symmetric matrix Q of quadratic coefficients.
        k: The number of nodes to select.
        timeout: The maximum time in seconds for the solver.

    Returns:
        A tuple containing:
            - The objective value of the solution.
            - The solver runtime.
            - A numpy array representing the solution vector.
            - The final MIP gap.
    """
    g = None
    try:
        # Get upperbound and lowerbound
        ubi = [k * max(matrix[i, j] for j in range(n)) for i in range(n)]

        lbi = [k * min(matrix[i, j] for j in range(n)) for i in range(n)]

        # Lowerbound for all variables
        lowerbound = min(0, min(lbi))

        # Setup model
        g = Model("Linear2")

        # Set time limit
        g.setParam("TimeLimit", timeout)
        g.setParam("OutputFlag", 0)

        # Get variables
        x_g = g.addVars(n, vtype=GRB.BINARY, name="x_g")

        # Get auxiliary variables
        w = g.addVars(n, vtype=GRB.CONTINUOUS, name="w", lb=lowerbound)

        # Select exactly k nodes
        g.addConstr(gp.quicksum(x_g[i] for i in range(n)) == k)

        # Constraint for w, x, ub where w_i <= x_i * ubi[i]
        g.addConstrs(w[i] <= x_g[i] * ubi[i] for i in range(n))

        # Constraint for w, x, lb where w_i >= x_i * lbi[i]
        g.addConstrs(w[i] >= x_g[i] * lbi[i] for i in range(n))

        # Constraint for w, x, q, lb where w_i <= sum(x_j * q_ij) - lbi[i] * (1 - x_i)
        g.addConstrs(
            w[i]
            <= gp.quicksum(x_g[j] * matrix[i, j] for j in range(n))
            - lbi[i] * (1 - x_g[i])
            for i in range(n)
        )

        # Constraint for w, x, q, ub
        g.addConstrs(
            w[i]
            >= gp.quicksum(x_g[j] * matrix[i, j] for j in range(n))
            - ubi[i] * (1 - x_g[i])
            for i in range(n)
        )

        # Add objective function
        g.setObjective(
            0.5
            * (
                gp.quicksum(w[i] for i in range(n))
                + gp.quicksum(matrix[i, i] * x_g[i] for i in range(n))
            ),
            GRB.MAXIMIZE,
        )

        # Solve
        g.optimize()

        return g.ObjVal, g.Runtime, np.array([v.X for v in x_g.values()]), g.MIPGap
    except gp.GurobiError as e:
        print("Error:", e)
        return None, None, None, None
    finally:
        if g:
            # Dispose of the model to free resources
            g.dispose()

## <span style="color:yellow; background-color:green;">Comparision</span>. We compare the performances of three different methods

### Comparision on MIPGap for a fixed time solve

In [ ]:
# Define the problem sizes (number of nodes) to test
n = [20, 40, 50, 60, 80, 90, 100]

# Initialize a dictionary to store the results from the comparison
comparision = {
    "n": [],
    "Solver": [],
    "Runtime": [],
    "MIPGap": [],
    "Optimal Value": [],
}
# Set an overall time limit for each solver to ensure a fair comparison
time_limit = 10

# Solve for each problem size
for s in n:

    # For each size 's', select half of the nodes
    k = math.floor(s / 2)

    # Generate a new random symmetric matrix for the current problem size
    matrix = create_matrix(s)

    # --- Solve the problem using the three different methods ---
    # 1. Gurobi's native QP solver
    qp_obj, qp_runtime, _, qp_mipgap = qp_model(
        n=s,
        matrix=matrix,
        k=k,
        timeout=time_limit,
    )

    # 2. First linearisation technique
    first_linear_obj, first_linear_runtime, _, first_linear_mipgap = first_linear(
        n=s,
        matrix=matrix,
        k=k,
        timeout=time_limit,
    )

    # 3. Second linearisation technique
    second_linear_obj, second_linear_time, _, second_linear_mipgap = second_linear(
        n=s,
        matrix=matrix,
        k=k,
        timeout=time_limit,
    )

    # --- Store the results for this problem size ---
    comparision["n"].extend(np.repeat(s, 3))
    comparision["Solver"].extend(["(QP) Solver", "First Linear", "Second Linear"])
    comparision["Runtime"].extend(
        [qp_runtime, first_linear_runtime, second_linear_time]
    )
    comparision["MIPGap"].extend([qp_mipgap, first_linear_mipgap, second_linear_mipgap])
    comparision["Optimal Value"].extend([qp_obj, first_linear_obj, second_linear_obj])

# Convert the results dictionary to a pandas DataFrame
df_comparision = pd.DataFrame(comparision)

# Display the comparison table
display(df_comparision)

Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 10
Set parameter TimeLimit to v

,n,Solver,Runtime,MIPGap,Optimal Value
0,20,(QP) Solver,0.058668,0.000000,319.0
1,20,First Linear,0.042029,0.000000,319.0
2,20,Second Linear,0.026712,0.000000,319.0
3,40,(QP) Solver,6.689836,0.000000,1179.5
4,40,First Linear,10.002871,0.029250,1179.5
5,40,Second Linear,10.006522,0.031793,1179.5
6,50,(QP) Solver,10.002272,0.473410,1824.0
7,50,First Linear,10.002741,0.088542,1824.0
8,50,Second Linear,10.009742,0.067982,1824.0
9,60,(QP) Solver,10.001536,0.561283,2541.5


### Visualise the performance

In [13]:
fig = px.line(
    data_frame=df_comparision,
    y="MIPGap",
    x="n",
    color="Solver",
    template="none",
    height=600,
    markers=True,
    title="Comparision of each algorithm's MIPGap, provided the same runtime.",
)
# fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()